In [1]:
import langchain
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from tqdm.autonotebook import tqdm, trange
from langchain.prompts import PromptTemplate

from langchain.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

import os
import warnings
from getpass import getpass

warnings.filterwarnings("ignore")

C:\Users\Evoortsolutions\AppData\Local\Temp\ipykernel_15912\2554702718.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
#add website data
URL = ["https://www.geeksforgeeks.org/stock-price-prediction-project-using-tensorflow/",
       "https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/"]
 
#load the data
data = WebBaseLoader(URL)
#extract the content
content = data.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
chunking = text_splitter.split_documents(content)

In [4]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
 
import os
from getpass import getpass
 
# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token
 
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name = "BAAI/bge-base-en-v1.5"
)

········


In [5]:
vectorstore = Chroma.from_documents(chunking,embeddings)

In [6]:
retriever = vectorstore.as_retriever(search_type='mmr',search_kwargs={"k":5})

In [7]:
query = "What is Recurrent Neural Networks?"
docs_rel = retriever.get_relevant_documents(query)
print(docs_rel[1])

page_content='Recurrent Neural Networks in TensorFlow\nRecurrent Neural Network is different from Convolution Neural Network and Artificial Neural Network. A Neural Network is basically known to be trained to learn deep features to make accurate predictions. Whereas Recurrent Neural Network works in such a way that there is feedback between each node which stores information in the cell state. In simple words, Recurrent Neural Networks are the process of backpropagating through time.' metadata={'description': 'A Computer Science portal for geeks. It contains well written, well thought and well explained computer science and programming articles, quizzes and practice/competitive programming/company interview Questions.', 'language': 'en-US', 'source': 'https://www.geeksforgeeks.org/training-of-recurrent-neural-networks-rnn-in-tensorflow/', 'title': 'Training of Recurrent Neural Networks (RNN) in TensorFlow - GeeksforGeeks'}


In [28]:
prompt_template = """
You are an intelligent assistant designed to provide answers based on the provided context. Follow these instructions:

***instructions***
1. Contextual Understanding:
   - Carefully read and understand the given context.
   - Use only the information provided in the context to generate your answer.
   
2. Answer Generation:
   - If the question is directly related to the context, provide a detailed and helpful answer.
   - If the information needed to answer the question is not mentioned in the context, respond with "I don't know."

Don't print anything extra use only answer as a response.

Provided Context:
{context}

Question:
{question}

Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [45]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    base_url='http://localhost:11434',
    model='llama3',
)

In [46]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
 
model = HuggingFaceHub(repo_id="google/flan-t5-base",
                       model_kwargs={"temperature":0.2,
                                     "max_new_tokens":240,
                                     "max_length":128
                                    })

# Define RetrievalQA chain
chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":PROMPT},
    chain_type="stuff"
)

In [43]:
query = "What is RNN?"
response = chain.invoke(query)
print(response['result'])

Recurrent Neural Networks (RNNs) are a class of neural networks that are particularly effective for sequential data. Unlike traditional feedforward neural networks, RNNs have connections that form directed cycles, allowing them to maintain a hidden state that can capture information from previous inputs.Recurrent Neural Networks (RNNs) are a class of neural networks that are particularly effective for sequential data. Unlike traditional feedforward neural networks, RNNs have connections that form directed cycles, allowing them to maintain a hidden state that can capture information from previous inputs.


In [47]:
query = "What is Selenium?"
response = chain({"query": query})
print(response['result'])

I don't know.


In [48]:
query = "What is virtual functions in cpp?"
response = chain({"query": query})
print(response['result'])

I don't know.


In [49]:
query = "What is CNNs?"
response = chain({"query": query})
print(response['result'])

Convolutional Neural Networks (CNNs).


In [50]:
query = "What is RAG?"
response = chain({"query": query})
print(response['result'])

I don't know.


In [51]:
query = "How RNNs trained explain me training in detail?"
response = chain({"query": query})
print(response['result'])

Now that the data is ready, the next step is building a Simple Recurrent Neural network. Before training with SimpleRNN, the data is passed through the Embedding layer to perform the equal size of Word Vectors. The model's training process involves passing the data through the layers in sequence: Embedding → RNN → Output.
